# Computer Networks - ECS252 - Spring 2022

## Chapter 1: Reading Assignment

### Section 1.1 What is in the Internet? 
A quick read
1. The "Nuts and bolt view." Whats are the different componnets that make up the Internet 
2. The "Service view." What are the services and applications supportted by the Internet 
3. What is a protocol 

### Section 1.2  Network Edge
A quick read and knowlegdge of the some of the key terms

1. What are end-systems and their roles? 
2. What is an access network? Some common types of access networks

### Section 1.3 Network Core 
Understanding the difference  between packet switching and circuit switching. 

1. What are the main features of packet switching? 
2. What is statistical multiplexing?
3. What is a store forward network? 
4. What is circuit switching? 
5. A comparison between packet switchng and circuit switching. 


### Section 1.4 Delay, Loss, and Throughput in Packet Switched Networks
Important section

1. What are the different delays? 
2. What is propagation delay? 
3. What is transmission delay? 
4. What is queueing delay? 
5. What is processing delay? 
6. Whar is throughput? 

### Section 1.5
A quick read

1. What are the protocol layers in the Internet protocol stack?
2. What are the functions of the different layers? 

### Section 1.6
A quick read

1. What are somne major types of attack on  the network? 

## Queueing Delay

Packet switching uses statistical multiplexing. As a result, a times the incoming paket rate to a router interface  can be larger than the outgoing rate. As a result, packets need to be queued. This results in queueing delay. For example wheh you arrive at a bank, all the tellers may be serving other customers and you join the queue. 

For single server system with infinite buffer capacity, packet arrivals following a Poisson process with rate $\lambda$ pkts/sec and service time following and Exponential distiibution with rate $\mu$ pkts/sec, we derived the mean queue length $E(N)$ whic is given by 
\begin{equation}
E(N) = \frac{\rho}{1 - \rho}
\end{equation}
where $\rho = \frac{\lambda}{\mu}$ and is called the traffic intensity. We used Markov Chains to derive the result. For the steady state probabilities of the Markov Chain to exist,  $\rho < 1$. 

Using Little's Law we can find the mean delay $E(D)$ which is given by 
\begin{equation}
E(D) = \frac{1/\mu}{1 - \rho}
\end{equation}

## Simulation of a Single Server M/M/1 Queue

The following code is a discrete event simulation of a single server M/M/1 Queue. It uses the python module simpy which is a powerful framework for any discrete event simulation. 

Couple of notes about  the simulation:

1. Since the packet arrivals follow a Poisson process with rate $\lambda$,  the packet inter-arrival time follow an Exponential distribution with parameter $\lambda$ pkts/sec. The expovariate function in numpy.random module is used to generate samples for the packet inter-arrival times. 

2. For this simulation you really don't need the packet class the queue (as a list). It is enough to just keep a count of the number of packets. 

In [ ]:
import random
import simpy
import math


# First  define some global variables
class G:
    RANDOM_SEED = 33
    SIM_TIME = 100000
    MU = 1
    LONG_SLEEP_TIMER = 1000000000

        
class Server_Process(object):
    def __init__(self, env, Server_Idle_Periods, Packet_Delay):
        self.env = env
        self.len = 0  # total number of packets in the system 
        self.queue = []  # the buffer
        self.server_busy = False  
        # this will the area 
        self.prevous_time = 0
        self.sum_time_length = 0
        self.delay = 0
        self.start_idle_time = 0
        self.Server_Idle_Periods = Server_Idle_Periods
        self.Packet_Delay = Packet_Delay
        self.action = env.process(self.run())
            
    def run(self):
        #print("TX-process started")
        while True: 
            try: 
                yield self.env.timeout(G.LONG_SLEEP_TIMER)
            except simpy.Interrupt:
                while self.len > 0:
                    packet = self.queue.pop()
                    yield self.env.timeout(random.expovariate(G.MU))
                    self.delay = self.env.now - packet.arrival_time
                    self.Packet_Delay.addNumber(self.delay)
                    self.sum_time_length += (self.env.now - self.prevous_time)*self.len
                    self.prevous_time = self.env.now
                    self.len -= 1
                self.server_busy = False
                self.start_idle_time = self.env.now


class Arrival_Process(object): 
    def __init__(self, env, arrival_rate, server_process, Server_Idle_Periods, Packet_Delay):
        
         self.env = env
         self.arrival_rate = arrival_rate
         self.server_process = server_process
         self.Server_Idle_Periods = Server_Idle_Periods
         self.Packet_Delay = Packet_Delay
         self.packet_number = 0
         self.action = env.process(self.run())
        
    def run(self):
        # packet arrivals 
        #print("Arrival Process Started")
        
        while True:
             # Infinite loop for generating packets
            yield self.env.timeout(random.expovariate(self.arrival_rate))
            
            self.server_process.sum_time_length += (self.env.now - self.server_process.prevous_time)*self.server_process.len
            self.server_process.prevous_time = self.env.now
            
            #create and enque new packet
            self.packet_number += 1
            arrival_time = self.env.now  
            new_packet = Packet(self.packet_number,arrival_time)
            self.server_process.len += 1
            #print(self.server_process.len)
            self.server_process.queue.append(new_packet)
            
            
            if self.server_process.server_busy == False:
                self.server_process.server_busy = True
                idle_period = self.env.now - self.server_process.start_idle_time

                self.Server_Idle_Periods.addNumber(idle_period)
                self.server_process.action.interrupt()

        

class Packet:
    def __init__(self, identifier, arrival_time):
        self.identifier = identifier
        self.arrival_time = arrival_time


class StatObject(object):
    def __init__(self):
        self.dataset =[]

    def addNumber(self,x):
        self.dataset.append(x)
    def sum(self):
        n = len(self.dataset)
        sum = 0
        for i in self.dataset:
            sum = sum + i
        return sum
    def mean(self):
        n = len(self.dataset)
        sum = 0
        for i in self.dataset:
            sum = sum + i
        return sum/n
    def maximum(self):
        return max(self.dataset)
    def minimum(self):
        return min(self.dataset)
    def count(self):
        return len(self.dataset)
    def median(self):
        self.dataset.sort()
        n = len(self.dataset)
        if n//2 != 0: # get the middle number
            return self.dataset[n//2]
        else: # find the average of the middle two numbers
            return ((self.dataset[n//2] + self.dataset[n//2 + 1])/2)
    def standarddeviation(self):
        temp = self.mean()
        sum = 0
        for i in self.dataset:
            sum = sum + (i - temp)**2
        sum = sum/(len(self.dataset) - 1)
        return math.sqrt(sum)


def main():
    print("Simple queue system model:mu = {0}".format(G.MU))
    print ("{0:<9} {1:<9} {2:<9} {3:<9} {4:<9} {5:<9} {6:<9} {7:<9} {8:<9} {9:<9}".format(
        "Lambda", "Count", "Min", "Max", "Mean", "Median", "Sd", "Utilization", "Mean Len", "Theoretica"))
    random.seed(G.RANDOM_SEED)
    for arrival_rate in [0.1, 0.5, 0.9]:
        env = simpy.Environment()
        Server_Idle_Periods = StatObject()
        Packet_Delay = StatObject()
        server_process = Server_Process(env,Server_Idle_Periods, Packet_Delay)
        arrival_process = Arrival_Process(env, arrival_rate, server_process, Server_Idle_Periods, Packet_Delay)
        env.run(until=G.SIM_TIME)
        
        rho = arrival_rate/G.MU
        print ("{0:<9.3f} {1:<9} {2:<9.3f} {3:<9.3f} {4:<9.3f} {5:<9.3f} {6:<9.3f} {7:<11.3f} {8:<11.3f} {9:<9.3f}".format(
            round(arrival_rate, 3),
            int(Packet_Delay.count()),
            round(Packet_Delay.minimum(), 3),
            round(Packet_Delay.maximum(), 3),
            round(Packet_Delay.mean(), 3),
            round(Packet_Delay.median(), 3),
            round(Packet_Delay.standarddeviation(), 3),
            round(1-Server_Idle_Periods.sum()/G.SIM_TIME, 3), 
            round(server_process.sum_time_length/G.SIM_TIME, 3),
            round(rho/(1 - rho), 3)))
    
if __name__ == '__main__': main()

# Project 1: Simulation Analysis of Aloha and Binary Exponential Backoff Algorithm 

## Ethernet Simulation  - Starter Code

This code gives the structure of my implementation. There are other ways as well to implement this. 
We all think differently. So go through it couple of times to get the picture of how I have designed it. 

In [ ]:
import simpy
import math
import numpy as np
import matplotlib.pyplot as plt


# First  define some global variables. You should change values
class G:
    RANDOM_SEED = 33
    SIM_TIME = 1000   # This should be large
    SLOT_TIME = 1
    N = 10
    ARRIVAL_RATES = [0.001, 0.002, 0.003, 0.006, 0.012, 0.024]  # Check the submission guidelines
    RETRANMISSION_POLICIES = ["pp", "op", "beb", "lb"]
    LONG_SLEEP_TIMER = 1000000000

        
class Server_Process(object):
    def __init__(self, env, dictionary_of_nodes, retran_policy, slot_stat):
        self.env = env
        self.dictionary_of_nodes = dictionary_of_nodes 
        self.retran_policy = retran_policy 
        self.slot_stat = slot_stat
        self.current_slot = 0
        self.action = env.process(self.run())
            
    def run(self):
        print("Server process started")
        
        while True: 
            # sleep for slot time
            yield self.env.timeout(G.SLOT_TIME)
            
            # Code to determine what happens to a slot and 
            # then update node variables accordingly based 
            # on the algorithm 
  

                    
        
class Node_Process(object): 
    def __init__(self, env, id, arrival_rate):
        
        self.env = env
        self.id = id
        self.arrival_rate = arrival_rate
        
        # Other state variables
        
        self.action = env.process(self.run())
        

    def run(self):
        # packet arrivals 
        print("Arrival Process Started:", self.id)
        
        # Code to generate the next packet and deal with it
        
        
        

class Packet:
    def __init__(self, identifier, arrival_time):
        self.identifier = identifier
        self.arrival_time = arrival_time


class StatObject(object):    
    def __init__(self):
        self.dataset =[]

    def addNumber(self,x):
        self.dataset.append(x)




def main():
    print("Simiulation Analysis of Random Access Protocols")
    random.seed(G.RANDOM_SEED)

    for retran_policy in G.RETRANMISSION_POLICIES: 
        for arrival_rate in G.ARRIVAL_RATES:
            env = simpy.Environment()
            slot_stat = StatObject()
            dictionary_of_nodes  = {} # I chose to pass the list of nodes as a 
                                      # dictionary since I really like python dictionaries :)
            
            for i in list(range(1,G.N+1)):
                node = Node_Process(env, i, arrival_rate)
                dictionary_of_nodes[i] = node
            server_process = Server_Process(env, dictionary_of_nodes,retran_policy,slot_stat)
            env.run(until=G.SIM_TIME)
            
            # code to determine throughput
        
        # code to plot 
           
    
if __name__ == '__main__': main()